In [1]:
import pandas as pd 
import numpy as np
import os 
import datetime
import geopandas as gpd
from collections import Counter   # 用來方便累加每個 chunk 的統計結果

In [2]:
# 00_setup_os處理函數
def create_folder(folder_name):
    """建立資料夾"""
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return os.path.abspath(folder_name)

def findfiles(filefolderpath, filetype='.csv', recursive=True):
    """
    尋找指定路徑下指定類型的檔案，並返回檔案路徑列表。

    Args:
        filefolderpath (str): 指定的檔案路徑。
        filetype (str, optional): 要尋找的檔案類型，預設為 '.csv'。
        recursive (bool, optional): 是否檢索所有子資料夾，預設為 True；反之為False，僅查找當前資料夾的所有file。

    Returns:
        list: 包含所有符合條件的檔案路徑的列表。
    """
    filelist = []

    if recursive:
        # 遍歷資料夾及其子資料夾
        for root, _, files in os.walk(filefolderpath):
            for file in files:
                if file.endswith(filetype):
                    file_path = os.path.join(root, file)
                    filelist.append(file_path)
    else:
        # 僅檢索當前資料夾
        for file in os.listdir(filefolderpath):
            file_path = os.path.join(filefolderpath, file)
            if os.path.isfile(file_path) and file.endswith(filetype):
                filelist.append(file_path)

    return filelist

# 01_資料預處理
def filter_ticket_data(filepath, 
                       selectdate_start, 
                       selectdate_end, 
                       outputfolder,
                       skiprows=1, 
                       chunksize=1000,
                        on_time_column = 'BoardingTime', 
                       off_time_column = 'DeboardingTime', 
                       infodate_column = 'InfoDate',):
    """
    分批讀取大型票證 CSV，依上車時間欄位做日期篩選後輸出新的 CSV。
    
    Parameters
    ----------
    filepath : str
        原始 CSV 路徑
    on_time_column : str
        上車時間欄位名稱
    off_time_column : str
        下車時間欄位名稱（保留未來擴充）
    selectdate_start : str
        篩選起始日期（YYYY-MM-DD）
    selectdate_end : str
        篩選結束日期（YYYY-MM-DD）
    outputfolder : str
        最終輸出 CSV 的資料夾路徑
    skiprows : int
        讀取 CSV 時跳過的列
    chunksize : int
        每批讀取筆數

    Returns
    -------
    outputpath : str
        最終輸出 CSV 的完整路徑
    """

    # 建立輸出資料夾（如不存在）
    os.makedirs(outputfolder, exist_ok=True)

    # 產生輸出檔名
    filename = os.path.basename(filepath).replace(
        ".csv", f"_{selectdate_start}_to_{selectdate_end}.csv"
    )
    outputpath = os.path.join(outputfolder, filename)

    # 日期轉 datetime
    start = pd.to_datetime(selectdate_start)
    end   = pd.to_datetime(selectdate_end)

    # 分批讀取
    chunks = pd.read_csv(filepath, skiprows=skiprows, chunksize=chunksize)
    first_chunk = True

    for chunk in chunks:
        # 轉成 datetime
        # chunk[on_time_column] = pd.to_datetime(chunk[on_time_column], errors='coerce')
        # chunk[off_time_column] = pd.to_datetime(chunk[off_time_column], errors='coerce')
        chunk[infodate_column] = pd.to_datetime(chunk[infodate_column], errors='coerce')

        # 日期篩選
        # mask = (
        #     ((chunk[on_time_column]  >= start) & (chunk[on_time_column]  <= end)) |
        #     ((chunk[off_time_column] >= start) & (chunk[off_time_column] <= end))
        # )    
        # mask = (chunk[on_time_column] >= start) & (chunk[on_time_column] <= end)
        mask = (chunk[infodate_column] >= start) & (chunk[infodate_column] <= end)
        filtered_chunk = chunk[mask]

        if filtered_chunk.empty:
            continue

        # 寫入 CSV
        filtered_chunk.to_csv(
            outputpath,
            mode='w' if first_chunk else 'a',
            header=first_chunk,
            index=False,
            encoding='utf-8-sig'
        )
        first_chunk = False

    return outputpath

def tickets_cleaning(
    tickets, 
    on_time_column='on_time_column', 
    off_time_column='off_time_column', 
    getonstop='GetOnStop', 
    getoffstop='GetOffStop', 
    getonseq='GetOnSeq', 
    getoffseq='GetOffSeq'):
    """
    清理票證資料，篩選出符合條件的票證並輸出統計結果。
    可以用於檢查票證資料的正確性。
    """
    # 原始票證數量
    original_count = len(tickets)

    # 建立篩選條件
    valid_conditions = (
        (tickets[on_time_column] < tickets[off_time_column]) &  # 上車時間早於下車時間
        (tickets[getonstop] != tickets[getoffstop]) &  # 上下車站不同
        (tickets[getonseq] < tickets[getoffseq])  # 上下車序正確
    )

    # 檢查每個條件的異常數量
    late_count = (tickets[on_time_column] >= tickets[off_time_column]).sum()
    same_stop_count = (tickets[getonstop] == tickets[getoffstop]).sum()
    seq_error_count = (tickets[getonseq] >= tickets[getoffseq]).sum()
    

    # 篩選出符合條件的票證
    cleaned_tickets = tickets[valid_conditions]
    canuse_count = len(cleaned_tickets)

    # 統計結果
    output = {
        '原始票證數量': original_count,
        '資料正常':canuse_count, 
        '資料異常 - 上車晚於下車': late_count,
        '資料異常 - 同站上下車': same_stop_count,
        '資料異常 - 上下車次序錯誤': seq_error_count
    }

    correctrate = round((canuse_count / original_count) * 100, 1)
    return cleaned_tickets, output, correctrate

def export_ticketcorrectrate(filename, output, correctrate, txt_path):

    # 運算時間
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 判斷檔案是否已存在
    file_exists = os.path.exists(txt_path)

    # 若檔案不存在 → 用 w (寫入 header)
    # 若檔案存在 → 用 a (不寫 header)
    mode = "a" if file_exists else "w"

    with open(txt_path, mode, encoding="utf-8") as f:

        # 如果是新檔案，寫入 header
        if not file_exists:
            f.write("filename,timestamp,key,value\n")

        # 寫入 output 每筆資料
        for key, value in output.items():
            f.write(f"{filename},{timestamp},{key},{value}\n")

        # 寫入正確率
        f.write(f"{filename},{timestamp},正確率,{correctrate}\n")

    print(f"TXT (CSV 格式) 已輸出：{txt_path}")

def get_stop_fromtickets(df):
    """
    從票證資料中提取所有上下車站點資訊，並合併成一個包含所有站點的 DataFrame。
    用於檢查票種的站點是否為可用的站點，因為有站點才有辦法核對到GIS。
    
    參數:
    df (DataFrame): 包含票證資料的 DataFrame，需包含上下車站點相關欄位。
    
    回傳:
    DataFrame: 包含所有上下車站點資訊的 DataFrame。
    """
    
     # 選取需要的欄位
    select_columns = ['Authority', 'OperatorNo',  
                    'RouteUID', 'RouteName', 'SubRouteUID', 'SubRouteName', 'Direction']
    boarding_stop_columns = ['BoardingStopUID', 'BoardingStopName', 'BoardingStopSequence']
    deboarding_stop_columns = ['DeboardingStopUID', 'DeboardingStopName', 'DeboardingStopSequence']

    # 取上車資料
    dfboarding =  df[select_columns + boarding_stop_columns]
    dfboarding[select_columns + boarding_stop_columns] = dfboarding[select_columns + boarding_stop_columns].fillna('-99')
    dfboarding.columns = dfboarding.columns.str.replace('Boarding', '')
    dfboarding['OnorOff'] = 'On'

    # 取下車資料
    dfdeboarding =  df[select_columns + deboarding_stop_columns]
    dfdeboarding[select_columns + deboarding_stop_columns] = dfdeboarding[select_columns+ deboarding_stop_columns].fillna('-99')
    dfdeboarding.columns = dfdeboarding.columns.str.replace('Deboarding', '')
    dfdeboarding['OnorOff'] = 'Off'
    # 合併上下車站點資料
    df_stops = pd.concat([dfboarding, dfdeboarding], ignore_index=True)
    
    df_stops = (
        df_stops
        .fillna(-99)
        .groupby(df_stops.columns.tolist())
        .size()
        .reset_index(name='Count')
    )

    return df_stops

# 02_資料分析處理

In [3]:
# 00_Setup 所有全域函數

# 1.) 設定篩選日期區間
selectdate_start = '2024-10-01'
selectdate_end = '2024-11-30'

# 2.) 建立輸出資料夾
selecttime_ticket_folder = create_folder(os.path.join(os.getcwd(), '..', '01_初步篩選整理票證', '01_指定時間區間票證資料')) # 建立01-01 指定時間區間票證資料夾
checkok_ticketfolder = create_folder(os.path.join(os.getcwd(), '..', '01_初步篩選整理票證', '02_過濾可用票證資料')) # 建立01-02 過濾可用票證資料夾
check_stopfolder = create_folder(os.path.join(os.getcwd(), '..', '01_初步篩選整理票證', '03_所有使用到的點位')) # 建立01-03 所有使用到的點位資料夾


In [4]:
# 預處理01: 指定時間區間票證資料切分
def split_ticket_with_day(selectdate_start, selectdate_end, outputfolder):
        orginal_ticket_files = [
                                r'D:\B-Project\2025\6800\Technical\12票證資料\2024_2025\公路客運電子票證資料(TO1A)\公路客運電子票證資料(TO1A).csv', 
                                r'D:\B-Project\2025\6800\Technical\12票證資料\2024_2025\新北市公車電子票證資料(TO1A)\新北市公車電子票證資料(TO1A).csv', 
                                r'D:\B-Project\2025\6800\Technical\12票證資料\2024_2025\桃園市公車電子票證資料(TO1A)\桃園市公車電子票證資料(TO1A).csv', 
                                r'D:\B-Project\2025\6800\Technical\12票證資料\2024_2025\桃園市公車電子票證資料(TO2A)\桃園市公車電子票證資料(TO2A).csv'
                                ]
        for file in orginal_ticket_files:
                output = filter_ticket_data(
                        filepath = file,
                        infodate_column = 'InfoDate',
                        selectdate_start = selectdate_start,
                        selectdate_end = selectdate_end,
                        outputfolder = outputfolder,
                        skiprows = 1,
                        chunksize = 1000
                        )
                print("輸出路徑：", output)
# split_ticket_with_day(selectdate_start, selectdate_end, selecttime_ticket_folder)

In [ ]:
# 預處理02: 指定時間區間票證資料切分
# def get_correct_tickets(selecttime_ticket_folder, checkok_ticketfolder):

selecttime_ticket_files = findfiles(selecttime_ticket_folder, filetype='.csv', recursive=False)
correctratelog_path = os.path.join(checkok_ticketfolder, '客運票證資料正確率記錄.txt')

chunksize = 10000   

for file in selecttime_ticket_files:

    print(f"\n=== 開始處理：{file} ===")

    # 統計資料累加器
    total_stat = Counter()

    # 輸出清洗後 CSV 的路徑
    cleaned_output_path = os.path.join(
        checkok_ticketfolder,
        os.path.basename(file).replace(".csv", "_cleaned.csv")
    )

    first_chunk = True  # 控制 header

    # 分批讀取整個檔案
    for chunk in pd.read_csv(file, chunksize=chunksize, encoding='utf-8-sig'):

        # 跑你自己的清洗函數
        cleaned_df, correct_stat_info, correctrate_chunk = tickets_cleaning(
            chunk,
            on_time_column='BoardingTime',
            off_time_column='DeboardingTime',
            getonstop='BoardingStopUID',
            getoffstop='DeboardingStopUID',
            getonseq='BoardingStopSequence',
            getoffseq='DeboardingStopSequence'
        )

        # 累加統計
        total_stat.update(correct_stat_info)

        # 將清洗後的 cleaned_df 分批寫入新 CSV
        if not cleaned_df.empty:
            cleaned_df.to_csv(
                cleaned_output_path,
                mode='w' if first_chunk else 'a',
                header=first_chunk,
                index=False,
                encoding='utf-8-sig'
            )
            first_chunk = False

    # -------- 整份 CSV 的整體正確率 --------
    original_count = total_stat.get('原始票證數量', 0)
    canuse_count   = total_stat.get('資料正常', 0)

    if original_count > 0:
        final_correctrate = round(canuse_count / original_count * 100, 2)
    else:
        final_correctrate = 0.0

    # -------- 寫入 TXT（CSV 格式） --------
    export_ticketcorrectrate(
        filename=file,
        output=dict(total_stat),
        correctrate=final_correctrate,
        txt_path=correctratelog_path
    )

    print(f"清洗後資料輸出：{cleaned_output_path}")

In [ ]:
# 預處理03: 確認所有站點的經緯度在TDX都可以被核對出來

files = findfiles(checkok_ticketfolder)
for file in files:
    df = pd.read_csv(file, encoding='utf-8-sig')
    stop = get_stop_fromtickets(df)
    stop['file_source'] = os.path.basename(file)

    outputfilename = os.path.join(check_stopfolder, os.path.basename(file).replace('.csv', '_stops.csv'))
    stop.to_csv(outputfilename, index=False, encoding='utf-8-sig')
    print(f"站點資料輸出：{outputfilename}")

In [ ]:
# 讀取路網
stop_gdf = gpd.read_file(r'D:\B-Project\2025\6800\Technical\12票證資料\其他分析資料\公路客運站點資料\公路客運站點資料.shp') 
dfstop = pd.read_csv(r'D:\B-Project\2025\6800\Technical\12票證資料\TicketAnalysis\01_初步篩選整理票證\03_所有使用到的點位\公路客運電子票證資料(TO1A)_2024-10-01_to_2024-11-30_cleaned_stops.csv')

# 第一次比對：比對StopUID與StopName
dfcount = pd.merge(dfstop, stop_gdf[['StopUID', 'StopName', 'Lon', 'Lat']].drop_duplicates(subset=['StopUID', 'StopName']), on = ['StopUID', 'StopName'], how = 'left')
total = dfstop['Count'].sum()
abnormal = dfcount[(dfcount['Lon'].isna()) | (dfcount['Lat'].isna())]['Count'].sum()


print('總共有幾筆資料', f"{total:,}")
print('沒有對應經緯度座標的資料異常數量', f"{abnormal:,}")
print('影響比例', f"{abnormal / total:.4%}")
print("============================")

# 第二次比對：只比對StopUID
dfcount_2ndround = dfcount[(dfcount['Lon'].isna()) | (dfcount['Lat'].isna())].copy()
dfcount_2ndround = pd.merge(dfcount_2ndround.drop(columns=['Lon', 'Lat']), stop_gdf[['StopUID', 'Lon', 'Lat', 'StopName']].drop_duplicates(subset=['StopUID']), on = ['StopUID'], how = 'left', suffixes=('', '_gdf'))
total_2ndround = dfcount_2ndround['Count'].sum()
abnormal_2ndround = dfcount_2ndround[(dfcount_2ndround['Lon'].isna()) | (dfcount_2ndround['Lat'].isna())]['Count'].sum()

print('第二次比對 - 總共有幾筆資料', f"{total_2ndround:,}")
print('第二次比對 - 沒有對應經緯度座標的資料異常數量', f"{abnormal_2ndround:,}")
print('第二次比對 - 影響比例', f"{abnormal_2ndround / total_2ndround:.4%}")
print('第二次比對 - 影響佔可用票證的原始比例', f"{abnormal_2ndround / total:.4%}")
print("============================")

# 合併
dfcount_final = pd.concat([dfcount[~((dfcount['Lon'].isna()) | (dfcount['Lat'].isna()))], dfcount_2ndround], ignore_index=True)

In [ ]:
# # 預處理04: 確認資料各票種、各路線、平假日、起點、迄點筆數
# files = findfiles(checkok_ticketfolder)
# file = files[2] # 我先只指定一個檔案來跑
# df = pd.read_csv(file, encoding='utf-8-sig')
# df['BoardingTime'] = pd.to_datetime(df['BoardingTime'], errors='coerce')
# df['DeboardingTime'] = pd.to_datetime(df['DeboardingTime'], errors='coerce')
# df['BoardinngDate'] = df['BoardingTime'].dt.date
# df['DeboardingDate'] = df['DeboardingTime'].dt.date
# df['BoardingHour'] = df['BoardingTime'].dt.hour
# df['DeboardingHour'] = df['DeboardingTime'].dt.hour

# groupbycolumns = ['HolderType', 
#                   'RouteUID', 'RouteName', 'SubRouteUID', 'SubRouteName',
#                   'BoardingStopUID', 'BoardingStopName', 'BoardingStopSequence', 'BoardinngDate', 'BoardingHour',
#                   'DeboardingStopUID', 'DeboardingStopName', 'DeboardingStopSequence', 'DeboardingDate', 'DeboardingHour']

# df[groupbycolumns] = df[groupbycolumns].fillna('-99')
# df_count = df.groupby(groupbycolumns).size().reset_index(name='Count')